# Deep SARSA for different environments

## 1. Import the libraries

In [2]:
import numpy as np
import matplotlib.pyplot as plt #plotting library
from matplotlib import animation #animated visualizations library
from collections import namedtuple, deque
#nametuple creates tuple subclasses with name fields, access elements by names instead of index
#deque (double-ended queue) for adding and removing elements from both ends
from tqdm import tqdm
#add progress bars to Python code for easy monitoring progress of loops and tasks
# %matplotlib inline
import gym #environments for agents
from datetime import datetime #manipulating dates and times
import pandas as pd #work with structured data
import torch #Pytorch supports tensor computations and neural networks
import torch.nn as nn #Pytorch supports building neural networks
import torch.nn.functional as Function
#common functions in neural network operations
    # Activation functions (ReLU, sigmoid, tanh)
    # Loss functions (cross_entropy, mse_loss)
    #Utility functions for tensor manipulation (softmax, dropout, batch_norm, etc.)
import torch.optim as optim #optimization algorithms for training neural networks
import random #generate random numbers/selections
from collections import namedtuple, deque
import itertools
# provides various functions for creating iterators and combining them for complex interators
# includes cycle, chain, zip, etc.

## 2. Neural network model for approximating Q-values

In [8]:
class QNetwork(nn.Module):
    #Actor Model

    def __init__(self, state_size, action_size, seed, fc1_units=64, fc2_units=64):
        #Parameters
#             state_size (int): Dimensionality of input state space
#             action_size (int): Dimensionality of output action space
#             seed (int): Random seed for reproducibility
#             fc1_units (int): Number of neurons (units) in first fully connected hidden layer
#             fc2_units (int): Number of neurons (unit) in second fully connected hidden layer

        #Initialization
        super(QNetwork, self).__init__()
        self.seed = torch.manual_seed(seed) # Set the random seed
        state_size = env.observation_space.shape[0]
        #Ensure random numbers generate are reproducible
        #Running same code with the same seed will produce the same sequence of random numbers

        # nn.Linear creates fully connected layers (input units, output units)
        self.fc1 = nn.Linear(state_size, fc1_units)
        self.fc2 = nn.Linear(fc1_units, fc2_units)
        self.fc3 = nn.Linear(fc2_units, action_size)

    def forward(self, state):
        # Network to map state with action values (Q-values)
        #inside method, input state pass through each layer
        #ReLU activation functions are applied to the outputs of hidden layers
#         print('state:', state.shape)
        #x output of 1 layer is input to the next
        x = Function.relu(self.fc1(state))
        x = Function.relu(self.fc2(x))
        return self.fc3(x)

## 3. Set the parameters

In [9]:
gamma = 1 #Discount factor
tau = 1 #Soft update target parameters, tau = 1 = copy completely
alpha = 0.005 #learning rate
update_every = 4 #How often to update the network
device = torch.device("cpu") #Device Initialization

## 4. Agent learning implementation

In [10]:
class Agent():
    def __init__(self, state_size, action_size, seed):
        self.state_size = state_size
        self.action_size = action_size
        self.seed = random.seed(seed)

        self.qnetwork_local = QNetwork(state_size, action_size, seed).to(device)
        #self.qnetwork_target = QNetwork(state_size, action_size, seed).to(device)
        self.optimizer = optim.Adam(self.qnetwork_local.parameters(), lr=alpha)
        self.t_step = 0

    def step(self, state, action, reward, next_state, next_action, done, state_size):
        self.t_step += 1
        self.learn(state, action, reward, next_state, next_action, done, state_size)

    def act(self, state, epsilon):
        if isinstance(state, np.ndarray):
            if state.dtype == np.object_:
                # Convert elements of the NumPy array to compatible types
                state = np.array([self.convert_to_compatible_type(elem) for elem in state])
            # Convert the modified NumPy array to a PyTorch tensor for neural network input
            state = torch.as_tensor(state, dtype=torch.float32).to(device)
        elif isinstance(state, list) or isinstance(state, tuple):
            # Convert the state to a NumPy array and handle ragged nested sequences
            state = self.handle_ragged_nested_sequence(state)
            # Convert the modified NumPy array to a PyTorch tensor for neural network input
            state = torch.as_tensor(state, dtype=torch.float32).to(device)
        elif isinstance(state, torch.Tensor):
            # Ensure the tensor is in the correct device and data type
            state = state.to(device)
        else:
            raise TypeError("Input state should be a NumPy array, list, tuple, or torch.Tensor.")

        # Ensure the state has the correct shape for the neural network input
        state = state.unsqueeze(0)  # Add a batch dimension

        self.qnetwork_local.eval()  # Evaluation mode
        with torch.no_grad():  # Disable gradient calculation when choosing action
            action_values = self.qnetwork_local(state)  # Pass preprocessed states into local Q-network
        self.qnetwork_local.train()  # Set local Q network back to training mode after inference is complete

        # Epsilon-greedy action selection
        if random.random() > epsilon:
            # Return action with highest Q-value
            action = np.argmax(action_values.cpu().data.numpy())
        else:
            # Select a random action
            action = random.choice(range(self.action_size))
        action = min(max(action, 0), self.action_size - 1) # Ensure action is within valid range

        return action

    def convert_to_compatible_type(self, elem):
        # Convert the element to a compatible type if it is a numpy.object_
        if isinstance(elem, str):
            return float(elem)
        elif isinstance(elem, np.ndarray):
            return elem.astype(np.float32)
        else:
            return float(elem)

    def handle_ragged_nested_sequence(self, state):
        if not state:
            return np.array([])  # Return an empty NumPy array if the state is empty
        else:
            # Convert dictionaries to lists of values
            state = [list(d.values()) if isinstance(d, dict) else d for d in state]
            # Determine the maximum length among all nested sequences
            max_length = max(len(sublist) for sublist in state)
            # Pad shorter sequences with zeros to make them of equal length
            state = [sublist + [0] * (max_length - len(sublist)) if len(sublist) < max_length else sublist for sublist in state]
            # Convert the nested sequence to a NumPy array
            state = np.array(state)
            return state

    def learn(self, state, action, reward, next_state, next_action, done, state_size):

        state_size = env.observation_space.shape[0]

        # Function to flatten ragged nested sequences into a flat array of compatible type
        def flatten_nested_sequence(elem):
            flattened = []
            for e in elem:
                if isinstance(e, (list, tuple)):
                    flattened.extend(flatten_nested_sequence(e))
                elif isinstance(e, np.ndarray):
                    if e.dtype == np.object_:
                        raise ValueError("Cannot handle np.ndarray of type numpy.object_")
                    else:
                        flattened.append(e)
                else:
                    flattened.append(e)
            return flattened

        # Convert state and next_state to appropriate types
        if isinstance(state, np.ndarray):
            state = flatten_nested_sequence(state)
            state = torch.as_tensor(state, dtype=torch.float32).to(device)
        elif isinstance(state, torch.Tensor):
            state = state.to(device)
        else:
            state = torch.zeros((1, state_size), dtype=torch.float32).to(device)

        if isinstance(next_state, np.ndarray):
            next_state = flatten_nested_sequence(next_state)
            next_state = torch.as_tensor(next_state, dtype=torch.float32).to(device)
        elif isinstance(next_state, torch.Tensor):
            next_state = next_state.to(device)
        else:
            next_state = torch.zeros((1, state_size), dtype=torch.float32).to(device)
        # Convert action, reward, and done flag to tensors
        #action = torch.as_tensor(action, dtype=torch.float32).to(device)
        reward = torch.as_tensor(reward, dtype=torch.float32).to(device)
        #done = torch.as_tensor(done, dtype=torch.bool).to(device)

        if state.size() == torch.Size([1, 8]):
            # Convert the second type of state to the first type
            state = state.view(-1)

        self.optimizer.zero_grad()
        if i_episode < 3 or i_episode > 1998:
            print("State: ", state)
            print("State size: ", state.size())
            print("Action: ", action)
            print("All Q_current): ", self.qnetwork_local(state))
        Q_current = self.qnetwork_local(state)[action]
        
        if i_episode < 3 or i_episode > 1998:    
            print("Q(current_action): ", Q_current)

        #print(Q_current.shape)
        
        if i_episode < 3 or i_episode > 1998:
            print("Next state: ", next_state)
            print("Next action:  ", next_action)
            print("All Q_next: ",self.qnetwork_local(next_state))
        
        Q_next = self.qnetwork_local(next_state)[next_action]
        
        if i_episode < 3 or i_episode > 1998:
            print("Q(next_state): ", Q_next)
            print("Reward", reward)
        
        Q_target = reward + gamma * Q_next * (1 - done)
        
        if i_episode < 3 or i_episode > 1998:
            print("Q(target_action): ", Q_target)
            print("Error (Q-target_action - Q_current_action): ", Q_target - Q_current)
            print(" ")

        loss = Function.mse_loss(Q_current, Q_target)
        loss.backward()
        self.optimizer.step()

        # #Update the target network
        # self.soft_update(self.qnetwork_local, self.qnetwork_target, tau)


    def soft_update(self, local_model, target_model, tau):
        #local_model: online model, actively being trained. Weights will be copied from here
        #target_model: use to generate target Q-values during training. Weights will be copied to here
        #tau: interpolation parameters determine rate at which parameters of target models are updated
        #small tau slower update, big tau faster update, less stable

        #function iterates over parameters of both target model and local model using zip
        for target_param, local_param in zip(target_model.parameters(), local_model.parameters()):
            #for each target param - local param pair, update target param by the formula
            # target_param = tau*local_param + (1-tau)*target_local
            target_param.data.copy_(tau*local_param.data + (1.0-tau)*target_param.data)


    # Function to simulate a model in an environment
    def simulate_model(env_name, model_path):
        # Load the environment
        env = gym.make(env_name)

        # Get environment parameters
        state_size = env.observation_space.shape[0]
        action_size = env.action_space.n

        # Initialize the agent
        agent = Agent(state_size, action_size, seed)

        # Load the model weights
        agent.qnetwork_local.load_state_dict(torch.load(model_path))
        agent.qnetwork_local.eval()

        # Simulate the model in the environment
        scores = []
        n_episodes = 100  # Number of episodes for simulation
        max_t = 1000  # Maximum number of timesteps per episode

        for i_episode in tqdm(range(1, n_episodes+1)):
            state = env.reset()
            score = 0

            for t in range(max_t):
                action = agent.act(state, epsilon = 0)
                step_result = env.step(action)
                next_state, reward, done, _ = step_result[:4]
                next_action = agent.act(next_state, epsilon)
                agent.step(state,action,reward,next_state,next_action, done, state_size)
                state = next_state
                action = next_action
                score = score + reward
                if done:
                    break

            scores.append(score)

        # Close the environment
        env.close()

        # Print average score
        print("Average score:", np.mean(scores))

## 5. Training parameters and environments

In [11]:
envs = ['LunarLander-v2', 'MountainCar-v0'] #list of environments ,
seeds = [1,37,42] #list of seeds , 37, 42
n_episodes = 2000 #number of training episodes
max_t = 1000 #maximum number of timesteps
epsilon_start = 1 #starting value of epsilon greedy
epsilon_end = 0.01 #minimum value of epsilon
epsilon_decay = 0.995 #rate at which epsilon decays

## 6. Training implementation

In [12]:
print("Algorithm: Deep SARSA" )
for i in envs:
    print("ENVIRONMENT:-----------", i)
    env = gym.make(i)
    res=[]

    for j in seeds:
        print("Seed = ", j)
        rewards = []
        aver_reward = []
        aver = deque(maxlen = 100)
        state_size = env.observation_space.shape[0]
        action_size=env.action_space.n
        agent = Agent(state_size, action_size, j)
        epsilon = epsilon_start

        for i_episode in tqdm(range(1, n_episodes+1)):
            if i_episode < 3 or i_episode > 1998:
                print("Iteration number: ", i_episode-1)
            state = env.reset()
            score = 0
            action = agent.act(state, epsilon)
            for t in range(max_t):

                step_result = env.step(action)
                next_state, reward, done, _ = step_result[:4]
                next_action = agent.act(next_state, epsilon)
                agent.step(state,action,reward,next_state,next_action, done, state_size)
                state = next_state
                action = next_action
                score = score + reward
                if done:
                    break

            aver.append(score)
            aver_reward.append(np.mean(aver))
            rewards.append(score)
            epsilon = max(epsilon_end, epsilon_decay*epsilon) # decrease epsilon

        # Save the model
        reward = "model/"+ "Seed" + str(j)  + i + "_" + "_" + str(n_episodes) + "_" + str(datetime.now().strftime("%Y%m%d%H%M%S"))
        torch.save(agent.qnetwork_local.state_dict(), reward + '.pt')

        # Append the average reward to the results
        res.append(aver_reward)
        print("----------------End Algorithm--------------------")

    fig = plt.figure()

    reward = 'plots/' + i + '_result' + str(datetime.now().strftime("%Y%m%d%H%M%S"))
    df = pd.DataFrame({str(seeds[0]): res[0], str(seeds[1]): res[1], str(seeds[2]): res[2]})  # Use seed as column labels
    df.to_csv(reward + '.csv')
    print("------------------------End Environment-------------------")

    plt.xlabel("Episode")
    plt.ylabel("Reward")

    # Plot rewards for each seed
    for seed in seeds:
        plt.plot(df[str(seed)], label='Seed ' + str(seed))

    plt.title('Learning Curve ' + i)

    # Insert the legends in the plot
    fig.legend(loc='lower right')
    fig.savefig(reward + '.png', dpi=100)





Algorithm: Deep SARSA
ENVIRONMENT:----------- LunarLander-v2
Seed =  1


  0%|                                               | 0/2000 [00:00<?, ?it/s]

Iteration number:  0
State:  tensor([0., 0., 0., 0., 0., 0., 0., 0.])
State size:  torch.Size([8])
Action:  0
All Q_current):  tensor([-0.0122,  0.0065, -0.0236, -0.0899], grad_fn=<AddBackward0>)
Q(current_action):  tensor(-0.0122, grad_fn=<SelectBackward0>)
Next state:  tensor([-6.2571e-04,  1.4014e+00, -3.1653e-02, -2.2485e-01,  7.2394e-04,
         7.0954e-03,  0.0000e+00,  0.0000e+00])
Next action:   3
All Q_next:  tensor([-0.0058,  0.0767, -0.0193, -0.1091], grad_fn=<AddBackward0>)
Q(next_state):  tensor(-0.1091, grad_fn=<SelectBackward0>)
Reward tensor(-2.0661)
Q(target_action):  tensor(-2.1753, grad_fn=<AddBackward0>)
Error (Q-target_action - Q_current_action):  tensor(-2.1631, grad_fn=<SubBackward0>)
 
State:  tensor([-6.2571e-04,  1.4014e+00, -3.1653e-02, -2.2485e-01,  7.2394e-04,
         7.0954e-03,  0.0000e+00,  0.0000e+00])
State size:  torch.Size([8])
Action:  3
All Q_current):  tensor([-0.0163,  0.0577, -0.0056, -0.0178], grad_fn=<AddBackward0>)
Q(current_action):  tenso

State:  tensor([-0.0040,  1.2992, -0.0266, -0.5054, -0.0268, -0.0745,  0.0000,  0.0000])
State size:  torch.Size([8])
Action:  3
All Q_current):  tensor([-0.0207, -0.0385,  0.1218,  0.0333], grad_fn=<AddBackward0>)
Q(current_action):  tensor(0.0333, grad_fn=<SelectBackward0>)
Next state:  tensor([-0.0042,  1.2872, -0.0145, -0.5328, -0.0329, -0.1231,  0.0000,  0.0000])
Next action:   1
All Q_next:  tensor([-0.0206, -0.0374,  0.1210,  0.0345], grad_fn=<AddBackward0>)
Q(next_state):  tensor(-0.0374, grad_fn=<SelectBackward0>)
Reward tensor(-2.1364)
Q(target_action):  tensor(-2.1737, grad_fn=<AddBackward0>)
Error (Q-target_action - Q_current_action):  tensor(-2.2071, grad_fn=<SubBackward0>)
 
State:  tensor([-0.0042,  1.2872, -0.0145, -0.5328, -0.0329, -0.1231,  0.0000,  0.0000])
State size:  torch.Size([8])
Action:  1
All Q_current):  tensor([-0.0264, -0.0385,  0.1312,  0.0257], grad_fn=<AddBackward0>)
Q(current_action):  tensor(-0.0385, grad_fn=<SelectBackward0>)
Next state:  tensor([-0.

All Q_next:  tensor([-0.0872, -0.2254,  0.7193, -0.2418], grad_fn=<AddBackward0>)
Q(next_state):  tensor(-0.2254, grad_fn=<SelectBackward0>)
Reward tensor(-2.3013)
Q(target_action):  tensor(-2.5266, grad_fn=<AddBackward0>)
Error (Q-target_action - Q_current_action):  tensor(-2.2864, grad_fn=<SubBackward0>)
 
State:  tensor([-0.0041,  1.0783,  0.0315, -0.7389, -0.1284, -0.2283,  0.0000,  0.0000])
State size:  torch.Size([8])
Action:  1
All Q_current):  tensor([-0.0939, -0.2173,  0.7605, -0.2703], grad_fn=<AddBackward0>)
Q(current_action):  tensor(-0.2173, grad_fn=<SelectBackward0>)
Next state:  tensor([-0.0039,  1.0611,  0.0214, -0.7651, -0.1378, -0.1880,  0.0000,  0.0000])
Next action:   2
All Q_next:  tensor([-0.0945, -0.2164,  0.7597, -0.2706], grad_fn=<AddBackward0>)
Q(next_state):  tensor(0.7597, grad_fn=<SelectBackward0>)
Reward tensor(-1.8356)
Q(target_action):  tensor(-1.0759, grad_fn=<AddBackward0>)
Error (Q-target_action - Q_current_action):  tensor(-0.8586, grad_fn=<SubBackwa

...

All Q_current):  tensor([-74.9464, -74.0462, -74.6336, -94.6804], grad_fn=<AddBackward0>)
Q(current_action):  tensor(-74.0462, grad_fn=<SelectBackward0>)
Next state:  tensor([ 0.6061,  0.2440,  0.9594, -1.2125, -0.2248,  0.3940,  0.0000,  0.0000])
Next action:   2
All Q_next:  tensor([-74.8569, -73.8788, -73.5812, -94.3476], grad_fn=<AddBackward0>)
Q(next_state):  tensor(-73.5812, grad_fn=<SelectBackward0>)
Reward tensor(0.5732)
Q(target_action):  tensor(-73.0079, grad_fn=<AddBackward0>)
Error (Q-target_action - Q_current_action):  tensor(1.0382, grad_fn=<SubBackward0>)
 
State:  tensor([ 0.6061,  0.2440,  0.9594, -1.2125, -0.2248,  0.3940,  0.0000,  0.0000])
State size:  torch.Size([8])
Action:  2
All Q_current):  tensor([-74.8444, -73.8013, -73.6632, -94.3348], grad_fn=<AddBackward0>)
Q(current_action):  tensor(-73.6632, grad_fn=<SelectBackward0>)
Next state:  tensor([ 0.6161,  0.2175,  0.9884, -1.1757, -0.2048,  0.4008,  0.0000,  0.0000])
Next action:   1
All Q_next:  tensor([-75.82

100%|████████████████████████████████████| 2000/2000 [24:15<00:00,  1.37it/s]

tensor(-83.5211, grad_fn=<AddBackward0>)
Error (Q-target_action - Q_current_action):  tensor(-2.2712, grad_fn=<SubBackward0>)
 
State:  tensor([ 0.7375, -0.1023,  0.9855, -1.1106,  0.0956,  0.5502,  0.0000,  0.0000])
State size:  torch.Size([8])
Action:  2
All Q_current):  tensor([ -83.2054,  -82.9346,  -82.1914, -104.4304], grad_fn=<AddBackward0>)
Q(current_action):  tensor(-82.1914, grad_fn=<SelectBackward0>)
Next state:  tensor([ 0.7476, -0.1263,  0.9817, -1.0676,  0.1235,  0.5562,  0.0000,  0.0000])
Next action:   2
All Q_next:  tensor([ -84.6341,  -84.6352,  -84.4381, -106.3003], grad_fn=<AddBackward0>)
Q(next_state):  tensor(-84.4381, grad_fn=<SelectBackward0>)
Reward tensor(-1.0005)
Q(target_action):  tensor(-85.4387, grad_fn=<AddBackward0>)
Error (Q-target_action - Q_current_action):  tensor(-3.2473, grad_fn=<SubBackward0>)
 
State:  tensor([ 0.7476, -0.1263,  0.9817, -1.0676,  0.1235,  0.5562,  0.0000,  0.0000])
State size:  torch.Size([8])
Action:  2
All Q_current):  tensor([

IndexError: list index out of range

<Figure size 640x480 with 0 Axes>

## 7. Demonstration with learned policy

In [ ]:
import gym
from tqdm import tqdm
import torch
import numpy as np

# Function to simulate a model in an environment
def simulate_model(env, agent, n_episodes=100, max_t=1000):
    scores = []

    for i_episode in tqdm(range(1, n_episodes+1)):
        state = env.reset()
        score = 0
        epsilon = 0

        for t in range(max_t):
            env.render()  # Move rendering here
            action = agent.act(state, epsilon = 0)
            step_result = env.step(action)
            next_state, reward, done, _ = step_result[:4]
            next_action = agent.act(next_state, epsilon = 0)
            agent.step(state,action,reward,next_state,next_action, done, state_size)
            state = next_state
            action = next_action
            score = score + reward
            if done:
                break
        print(score)
        scores.append(score)

    # Close the environment
    env.close()

    # Print average score
    print("Average score:", np.mean(scores))


env_name = 'LunarLander-v2'
model_path = 'model/Seed37_LunarLander-v2__DeepSARSA_4000_20240407085617.pt'
env = gym.make(env_name,  render_mode = 'human')

# Get environment parameters
state_size = env.observation_space.shape[0]
action_size = env.action_space.n

# Initialize the agent
agent = Agent(state_size, action_size, seed=37)

# Load the model weights
agent.qnetwork_local.load_state_dict(torch.load(model_path))
agent.qnetwork_local.eval()

# Simulate the model in the environment
simulate_model(env, agent)


## 8. Plot the shaded graph

In [ ]:
# Step 1: Read Data from CSV
df = pd.read_csv('plots/PriorityDDQN_MountainCar.csv')

# Step 2: Remove the first row (Episodes, Run 1, Run 2, Run 3) to keep only the rewards data
# df = df.drop(0)

# Step 3: Convert the remaining DataFrame to numeric values
df = df.apply(pd.to_numeric)

# Step 4: Select only the three late columns (Run 1, Run 2, Run 3)
later_columns = df.iloc[:, 1:4]

# Step 5: Calculate Mean and Standard Deviation for the three later columns
mean_values = later_columns.mean(axis=1)
std_values = later_columns.std(axis=1)
print("Standard deviation: ", std_values)
print("Mean: ", mean_values)

# Step 6: Plot the Data
plt.figure(figsize=(12, 6))
plt.plot(mean_values, label='Mean Sum of Rewards', color = "red")
plt.fill_between(mean_values.index, mean_values - std_values, mean_values + std_values, alpha=0.5, label='Standard Deviation', color = "orange")
plt.xlabel('Episode')
plt.ylabel('Sum of Rewards')
plt.title('Shaded Plot of Sum of Rewards with Mean and Standard Deviation for Priority Double Deep Q-Learning Algorithm, Mountain Car')
plt.savefig('plots/shaded_plot_PriorityDDQN_MountainCar.png')
plt.legend()
plt.show()

In [2]:
pip install ufal.pybox2d